In [1]:
# basic stuff 
from scipy import stats, signal, io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# from google.colab import files
import os
import sklearn
import time

# torch stuff
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torch import nn
import torch.nn.functional as F
!pip install gpytorch
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.likelihoods import MultitaskGaussianLikelihood
from gpytorch.kernels import RBFKernel
from gpytorch.mlls import ExactMarginalLogLikelihood

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Miller_Lab/FIU/PopFRData/')
sys.path.append('/content/drive/MyDrive/Miller_Lab/FIU/xds_python/')
sys.path.append('/content/drive/MyDrive/Miller_Lab/FIU/custom_functions/')

import joint_angles
import data_loading
import train_test
import r2_pr2
import models
import visualization

In [4]:
dataset_dict_fr = data_loading.load_datasets('/content/drive/My Drive/Miller_Lab/FIU/PopFRData/processed_shuffled_opensim_datasets/fullyrestrained')

In [26]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_tasks):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_tasks)
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=num_tasks, rank=1)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

In [21]:
train_x, train_y = data_loading.convert_osim_dataset_to_array(dataset_dict_fr['20220309']['Joint Angles']['Train'])
test_x, test_y = data_loading.convert_osim_dataset_to_array(dataset_dict_fr['20220309']['Joint Angles']['Test'])

train_x, train_y= torch.from_numpy(train_x), torch.from_numpy(train_y)
test_x, test_y= torch.from_numpy(test_x), torch.from_numpy(test_y)

In [10]:
train_x.shape, train_y.shape

(torch.Size([13895, 24]), torch.Size([13895, 96]))

In [22]:
dom = visualization.depth_of_modulation(train_y)

In [23]:
sorted_indices = np.argsort(dom)
top_30_indices = sorted_indices[-1:]
train_y_best_neurons = train_y[:1000,top_30_indices]

In [37]:
train_y_trunc = train_y[:5000,:]
train_x_trunc = train_x[:5000,:]

In [39]:
train_x_model = train_x_trunc
train_y_model = train_y_trunc

In [40]:
num_tasks = train_y_model.shape[1]
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_tasks)
model = MultitaskGPModel(train_x_model, train_y_model, likelihood, num_tasks)

training_iterations = 70

model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
losses = []
num_epochs = 70

In [41]:
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(train_x_model)
    loss = -mll(output, train_y_model)
    loss.backward()
    optimizer.step()

    print('Iter %d/%d - Loss: %.3f' % (epoch + 1, num_epochs, loss.item()))

Iter 1/70 - Loss: 138.062
Iter 2/70 - Loss: 126.411
Iter 3/70 - Loss: 114.997
Iter 4/70 - Loss: 103.984
Iter 5/70 - Loss: 93.544
Iter 6/70 - Loss: 83.784
Iter 7/70 - Loss: 74.759
Iter 8/70 - Loss: 66.491
Iter 9/70 - Loss: 58.988
Iter 10/70 - Loss: 52.251
Iter 11/70 - Loss: 46.263
Iter 12/70 - Loss: 40.997
Iter 13/70 - Loss: 36.408
Iter 14/70 - Loss: 32.441
Iter 15/70 - Loss: 29.028
Iter 16/70 - Loss: 26.106
Iter 17/70 - Loss: 23.610
Iter 18/70 - Loss: 21.482
Iter 19/70 - Loss: 19.669
Iter 20/70 - Loss: 18.123
Iter 21/70 - Loss: 16.803
Iter 22/70 - Loss: 15.675
Iter 23/70 - Loss: 14.708
Iter 24/70 - Loss: 13.878
Iter 25/70 - Loss: 13.164
Iter 26/70 - Loss: 12.549
Iter 27/70 - Loss: 12.020
Iter 28/70 - Loss: 11.563
Iter 29/70 - Loss: 11.166
Iter 30/70 - Loss: 10.821
Iter 31/70 - Loss: 10.519
Iter 32/70 - Loss: 10.254
Iter 33/70 - Loss: 10.019
Iter 34/70 - Loss: 9.809
Iter 35/70 - Loss: 9.621
Iter 36/70 - Loss: 9.451
Iter 37/70 - Loss: 9.296
Iter 38/70 - Loss: 9.155
Iter 39/70 - Loss: 9.0

In [ ]:
directory = '/content/drive/My Drive/Miller_Lab/FIU/GP/'
model_name = 'GP_20210712'
if os.path.exists(directory) == False:
    os.makedirs(directory)
torch.save(model.state_dict(), directory+model_name)

In [ ]:
model.eval()
likelihood.eval()

# Initialize plots
f, (y1_ax, y2_ax) = plt.subplots(1, 2, figsize=(8, 3))

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = torch.linspace(0, 1, 51)
    predictions = likelihood(model(test_x))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()

In [ ]:
model.eval()
likelihood.eval()

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_train_pred = likelihood(model(train_x_model))
    train_lower, train_upper = observed_train_pred.confidence_region()
    test_mean = observed_train_pred.mean.numpy()

/usr/local/lib/python3.9/dist-packages/gpytorch/models/exact_gp.py:283: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [11]:
import math
train_x = torch.linspace(0, 1, 100)

train_y = torch.stack([
    torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,
    torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,
], -1)

In [12]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=2
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2)
model = MultitaskGPModel(train_x, train_y, likelihood)

In [15]:
# Find optimal model hyperparameters
training_iterations = 50
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()

Iter 1/50 - Loss: 1.233
Iter 2/50 - Loss: 1.195
Iter 3/50 - Loss: 1.154
Iter 4/50 - Loss: 1.110
Iter 5/50 - Loss: 1.066
Iter 6/50 - Loss: 1.022
Iter 7/50 - Loss: 0.978
Iter 8/50 - Loss: 0.934
Iter 9/50 - Loss: 0.892
Iter 10/50 - Loss: 0.850
Iter 11/50 - Loss: 0.810
Iter 12/50 - Loss: 0.771
Iter 13/50 - Loss: 0.732
Iter 14/50 - Loss: 0.694
Iter 15/50 - Loss: 0.656
Iter 16/50 - Loss: 0.617
Iter 17/50 - Loss: 0.578
Iter 18/50 - Loss: 0.538
Iter 19/50 - Loss: 0.499
Iter 20/50 - Loss: 0.459
Iter 21/50 - Loss: 0.420
Iter 22/50 - Loss: 0.382
Iter 23/50 - Loss: 0.345
Iter 24/50 - Loss: 0.308
Iter 25/50 - Loss: 0.273
Iter 26/50 - Loss: 0.240
Iter 27/50 - Loss: 0.208
Iter 28/50 - Loss: 0.177
Iter 29/50 - Loss: 0.148
Iter 30/50 - Loss: 0.121
Iter 31/50 - Loss: 0.097
Iter 32/50 - Loss: 0.074
Iter 33/50 - Loss: 0.055
Iter 34/50 - Loss: 0.038
Iter 35/50 - Loss: 0.023
Iter 36/50 - Loss: 0.012
Iter 37/50 - Loss: 0.002
Iter 38/50 - Loss: -0.004
Iter 39/50 - Loss: -0.008
Iter 40/50 - Loss: -0.011
Iter 4